In [10]:
# %pip install tqdm
# %pip install pyedflib

import pyedflib
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
from tqdm import tqdm
import warnings
import torch
import pyedflib
import scipy.signal as signal
import h5py
from scipy import signal, interpolate
from scipy.stats import zscore

In [2]:
plt.style.use('bmh')

In [11]:
warnings.simplefilter(action='ignore', category=FutureWarning)

In [3]:
# Print out annotations of signals in EDF file
def edf_signal_labels(edf_file_path):
    signal_file = pyedflib.EdfReader(edf_file_path)
    signal_labels = signal_file.getSignalLabels()
    signal_file._close()

    return signal_labels


signal_labels_0001 = edf_signal_labels('/run/media/u2212061/DISK_YUAN/CS913/mesa/polysomnography/edfs/mesa-sleep-0001.edf')

print(signal_labels_0001)

['EKG', 'EOG-L', 'EOG-R', 'EMG', 'EEG1', 'EEG2', 'EEG3', 'Pres', 'Flow', 'Snore', 'Thor', 'Abdo', 'Leg', 'Therm', 'Pos', 'EKG_Off', 'EOG-L_Off', 'EOG-R_Off', 'EMG_Off', 'EEG1_Off', 'EEG2_Off', 'EEG3_Off', 'Pleth', 'OxStatus', 'SpO2', 'HR', 'DHR']


In [4]:
# Original frequency of PPG signal

def original_frequency(edf_file_path, signal_labels):
    signal_file = pyedflib.EdfReader(edf_file_path)
    signal_index = signal_labels.index('Pleth')
    sampling_rate = signal_file.getSampleFrequency(signal_index)

    signal_file._close()

    return sampling_rate


ppg_original_rate_0001 = original_frequency('/run/media/u2212061/DISK_YUAN/CS913/mesa/polysomnography/edfs/mesa-sleep-0001.edf', signal_labels_0001)

print(ppg_original_rate_0001)


256.0


In [4]:
# 1. Low-pass filtering removes high-frequency noise and prevents aliasing during down-sampling.
# We specifically used a low-pass filter as we wished to keep lower frequency components such as breathing and capillary modulation intact.
# A zero-phase 8th order low-pass Chebyshev Type II filter with a cutoff frequency of 8Hz and a stop-band attenuation of 40dB.
# https://docs.scipy.org/doc/scipy/reference/generated/scipy.signal.cheby2.html

def lowpass_filter(ppg_signal, original_freq, cutoff_freq=8, stopband_attenuation=40):
    # Wn
    nyquist_freq = original_freq * 0.5
    normalized_cutoff = cutoff_freq / nyquist_freq

    # cheby2 (second-order section)
    sos = signal.cheby2(N=8, rs=stopband_attenuation, Wn=normalized_cutoff, btype='lowpass', output='sos')
    filtered_signal = signal.sosfilt(sos, ppg_signal)

    return filtered_signal

In [4]:
# 2. Downsampling
# The filtered PPG was downsampled to 34.13Hz using linear interpolation, reducing the computational and memory requirements for ML.
# We choose a sampling rate of 34.13Hz as this resulted in 1024 (2^10) samples per 30s sleep-window. 
# By using a 2^n number we could maintain full temporal alignment of data with sleep-windows during ML pooling operations.

def downsample_signal(filtered_signal, original_freq, target_freq=34.13):
    original_time = np.arange(len(filtered_signal)) / original_freq
    new_time = np.arange(0, original_time[-1], 1/target_freq)
    f = interpolate.interp1d(original_time, filtered_signal, kind='linear')
    downsampled_ppg = f(new_time)

    return downsampled_ppg

In [6]:
# 3. Clean and standardize signals
# WAVPPG was cleaned by clipping values to three standard deviations and then standardized by subtracting the mean and dividing by the standard deviation.
# Normal distribution

def clean_and_standardize(downsampled_signal):
    mean = np.mean(downsampled_signal)
    std = np.std(downsampled_signal)
    
    clipped_signal = np.clip(downsampled_signal, mean - 3 * std, mean + 3 * std)
    
    standardized_signal = (clipped_signal - mean) / std

    return standardized_signal

In [12]:

def preprocess_ppg(ppg, original_fs=256):
    nyq = 0.5 * original_fs
    cutoff = 8 / nyq
    b, a = signal.cheb2ord(cutoff, cutoff + 0.1, 3, 40)
    sos = signal.cheby2(b, 40, cutoff, btype='lowpass', output='sos')
    filtered_ppg = signal.sosfiltfilt(sos, ppg)

    target_fs = 34.13
    original_time = np.arange(len(filtered_ppg)) / original_fs
    new_time = np.arange(0, original_time[-1], 1/target_fs)
    f = interpolate.interp1d(original_time, filtered_ppg, kind='linear')
    downsampled_ppg = f(new_time)

    mean = np.mean(downsampled_ppg)
    std = np.std(downsampled_ppg)
    clipped_ppg = np.clip(downsampled_ppg, mean - 3*std, mean + 3*std)

    wavppg = (clipped_ppg - np.mean(clipped_ppg)) / np.std(clipped_ppg)

    return wavppg

In [6]:
def interpolation(segment, original_fs=256, target_fs=34.13):
    time_old = np.arange(len(segment)) / original_fs
    time_new = np.arange(0, time_old[-1], 1/target_fs)
    f = interpolate.interp1d(time_old, segment, kind='quadratic', bounds_error=False, fill_value='extrapolate')
    segment = f(time_new)

    return segment

In [13]:
# Preprocessing
def find_edf_files(input_dir, output_dir):

    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    
    edf_file_names = []

    for filename in os.listdir(input_dir):
        if filename.endswith('.edf'):
            edf_file_names.append(filename)
    
    return edf_file_names

In [14]:
def read_ppg_from_edf(edf_file_path, signal_label='Pleth'):
    f = pyedflib.EdfReader(edf_file_path)
    signal_labels = f.getSignalLabels()
    signal_index = signal_labels.index(signal_label)
    ppg_signal = f.readSignal(signal_index)
    original_frequency = f.getSampleFrequency(signal_index)
    f._close()

    return ppg_signal, original_frequency

In [7]:
def convert_edf_to_txt_no_header(input_dir, output_dir):
    edf_file_names = find_edf_files(input_dir, output_dir)
    for name in tqdm(edf_file_names, desc='Converting EDF to TXT'):
        basename = name.split('.')[0] 
        try:
            path = os.path.join(input_dir, name)
            ppg_signal, original_freq = read_ppg_from_edf(path)
            # downsampled_signal = downsample_signal(ppg_signal, original_freq)
            interpolated_signal = interpolation(ppg_signal)
            
            df = pd.DataFrame(interpolated_signal) 
            
            output_file = os.path.join(output_dir, f'{basename}.txt')
            df.to_csv(output_file, index=False, sep='\t', header=None)
            # print(f'File {basename} converted successfully.')
        except Exception as e:
            print(f'Error converting file {basename}: {e}')

In [15]:
def convert_edf_to_txt(input_dir, output_dir):
    
    edf_file_names = find_edf_files(input_dir, output_dir)

    for name in tqdm(edf_file_names, desc='Converting EDF to TXT'):
        basename = name.split('.')[0]
        try:
            path = os.path.join(input_dir, name)
            ppg_signal, original_freq = read_ppg_from_edf(path)
            # filtered_signal = lowpass_filter(ppg_signal, original_freq)
            # downsampled_signal = downsample_signal(filtered_signal, original_freq)
            # downsampled_signal, downsampled_time = downsample_signal(filtered_signal, original_freq)
            # standardized_signal = clean_and_standardize(downsampled_signal)
            standardized_signal = preprocess_ppg(ppg_signal)

            df = pd.DataFrame(standardized_signal)
            # df = pd.DataFrame({'Time': downsampled_time, 'Pleth': standardized_signal})
            
            output_file = os.path.join(output_dir, f'{basename}.txt')
            df.to_csv(output_file, index=False, sep='\t', header=None)

            # print(f'File {basename} converted successfully.')

        except Exception as e:
            print(f'Error converting file {basename}: {e}.')

        
        


In [16]:
input_dir = '/run/media/u2212061/DISK_YUAN/913/mesa/polysomnography/edfs/'
output_dir = '/run/media/u2212061/DISK_YUAN/913/mesa/polysomnography/ppg_34/'

convert_edf_to_txt(input_dir, output_dir)

Converting EDF to TXT: 100%|██████████| 2055/2055 [1:41:20<00:00,  2.96s/it]


In [8]:
input_dir = '/run/media/u2212061/DISK_YUAN/913/mesa/polysomnography/edfs/'
output_dir = '/dcs/large/u2212061/raw_ppg_signal/'

convert_edf_to_txt_no_header(input_dir, output_dir)

Converting EDF to TXT: 100%|██████████| 2055/2055 [2:25:37<00:00,  4.25s/it]  


In [9]:
import xml.etree.ElementTree as ET

# Read XML and tag the sleeping stages to the signals
def parse_sleep_stages(input_dir, xml_filename):
    xml_file = input_dir + xml_filename
    tree = ET.parse(xml_file)
    root = tree.getroot()

    scored_events = root.find('.//ScoredEvents')
    if scored_events is None:
        print("ScoredEvents not found")
        return None

    sleep_stages = []

    for event in scored_events.iter('ScoredEvent'):
        event_type_element = event.find('EventType')
        event_type = event_type_element.text if event_type_element is not None else None

        if event_type == 'Stages|Stages':
            event_concept = event.find('EventConcept').text
            
            start_time = float(event.find('Start').text)

            duration = float(event.find('Duration').text)

            if 'Wake' in event_concept:
                stage = 0
            elif 'Stage 1 sleep' in event_concept or 'Stage 2 sleep' in event_concept:
                stage = 1
            elif 'Stage 3 sleep' in event_concept or 'Stage 4 sleep' in event_concept:
                stage = 2
            elif 'REM sleep' in event_concept:
                stage = 3
            else:
                continue

            sleep_stages.append({'Start': start_time, 
                                'Duration': duration, 
                                'Stage': stage})

    df_stage = pd.DataFrame(sleep_stages)

    return df_stage
    


In [10]:
def xml_to_txt_file(output_dir, xml_filename, df_stage):
    basename = xml_filename.split('.')[0]
    parts = basename.split('-')
    txt_filename = '-'.join(parts[:3]) + '.txt'

    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    df_stage.to_csv(output_dir + txt_filename, sep='\t', index=False)




In [11]:
def convert_xml_to_txt(input_dir, output_dir):
    xml_files = [file for file in os.listdir(input_dir) if file.endswith('.xml')]

    for xml_file in tqdm(xml_files, desc='Processing XML files'):
        xml_filename = xml_file
        df_stage = parse_sleep_stages(input_dir, xml_filename)
            
        xml_to_txt_file(output_dir, xml_filename, df_stage)


In [12]:
input_dir = '/run/media/u2212061/DISK_YUAN/CS913/mesa/polysomnography/annotations-events-nsrr/'
output_dir = '/run/media/u2212061/DISK_YUAN/CS913/mesa/polysomnography/xml_txt/'

convert_xml_to_txt(input_dir, output_dir)

Processing XML files: 100%|██████████| 2056/2056 [03:47<00:00,  9.05it/s]


In [17]:
def expand_labels_to_30s_window(input_dir, output_dir):

    for txt_file in tqdm(os.listdir(input_dir), desc='Expanding labels to 30s window'):
        df_xml = pd.read_csv(os.path.join(input_dir, txt_file), sep='\t')
        df_xml = pd.DataFrame(df_xml)

        total_time = df_xml['Start'].iloc[-1] + df_xml['Duration'].iloc[-1]
        num_segments = int(np.ceil(total_time / 30.0))

        labels = np.zeros(num_segments, dtype=int)

        for i, row in df_xml.iterrows():
            start_idx = int(row['Start'] // 30)
            end_idx = int((row['Start'] + row['Duration']) // 30)

            if end_idx > len(labels):
                end_idx = len(labels)
            
            labels[start_idx:end_idx] = row['Stage']
        
        if not os.path.exists(output_dir):
            os.makedirs(output_dir)
        
        # df_stage = pd.DataFrame({'Stage': labels})
        df_stage = pd.DataFrame(labels)
        df_stage.to_csv(output_dir + txt_file, sep='\t', index=False, header=None)




In [18]:
input_dir = '/run/media/u2212061/DISK_YUAN/913/mesa/polysomnography/xml_txt/'
output_dir = '/dcs/large/u2212061/raw_stage_ann/'

expand_labels_to_30s_window(input_dir, output_dir)

Expanding labels to 30s window: 100%|██████████| 2056/2056 [02:31<00:00, 13.58it/s]


In [17]:
# Padding zero and truncating to 1228800 for signal data 

def padding_and_truncating_signal_data(input_dir, output_dir, target_length):
    for txt_file in tqdm(os.listdir(input_dir), desc='Padding and Truncating to 10 hours'):
        # df_ppg = pd.read_csv(os.path.join(input_dir, txt_file), sep='\t')
        df_ppg = np.loadtxt(os.path.join(input_dir, txt_file), dtype='float')

        current_length = len(df_ppg)

        if target_length <= current_length:
            # df_ppg = df_ppg.iloc[:target_length]
            df_ppg = df_ppg[:target_length]
        else:
            padding_length = target_length - current_length
            # padding = np.zeros((padding_length, df_ppg.shape[1]))
            padding = np.zeros(padding_length)
            # df_ppg = pd.concat([df_ppg, pd.DataFrame(padding)], ignore_index=True)
            df_ppg = np.concatenate([df_ppg, padding])
        
        if not os.path.exists(output_dir):
            os.makedirs(output_dir)
        
        # df_ppg.to_csv(output_dir + txt_file, sep='\t', index=False, header=None)
        np.savetxt(output_dir + txt_file, df_ppg, fmt='%.18f')



In [18]:
input_dir = '/run/media/u2212061/DISK_YUAN/913/mesa/polysomnography/ppg_34/'
output_dir = '/dcs/large/u2212061/ppg_34_zero/'
target_length = 1228800

padding_and_truncating_signal_data(input_dir, output_dir, target_length)

Padding and Truncating to 10 hours: 100%|██████████| 2055/2055 [1:08:57<00:00,  2.01s/it]


In [47]:
# Padding -1 and truncating to 1200 for labels (stages)
def padding_and_truncating_stage_data(input_dir, output_dir, match_dir, target_length):
    for txt_file in tqdm(os.listdir(input_dir), desc='Padding and Truncating to 10 hours'):
        if txt_file in os.listdir(match_dir):

            # df_stage = pd.read_csv(os.path.join(input_dir, txt_file), sep='\t')
            df_stage = np.loadtxt(os.path.join(input_dir, txt_file), dtype='int')

            current_length = len(df_stage)

            if target_length <= current_length:
                # df_stage = df_stage.iloc[:target_length]
                df_stage = df_stage[:target_length]
            else:
                padding_length = target_length - current_length
                # Padding with -1
                # padding = np.full((padding_length, df_stage.shape[1]), int(-1))
                padding = np.full(padding_length, int(-1))
                # df_stage = pd.concat([df_stage, pd.DataFrame(padding)], ignore_index=True)
                df_stage = np.concatenate([df_stage, padding])

                
            if not os.path.exists(output_dir):
                os.makedirs(output_dir)
                
            # df_stage.to_csv(output_dir + txt_file, sep='\t', index=False, header=None)
            np.savetxt(output_dir + txt_file, df_stage, fmt='%d')


In [48]:
input_dir = '/run/media/u2212061/DISK_YUAN/CS913/mesa/polysomnography/xml_30_stage/'
output_dir = '/run/media/u2212061/DISK_YUAN/CS913/mesa/polysomnography/stage_30_minus/'
match_dir = '/run/media/u2212061/DISK_YUAN/CS913/mesa/polysomnography/ppg_34_zero/'
target_length = 1200

padding_and_truncating_stage_data(input_dir, output_dir, match_dir, target_length)

Padding and Truncating to 10 hours: 100%|██████████| 2056/2056 [06:35<00:00,  5.20it/s]


In [45]:
df_stage = pd.read_csv('/run/media/u2212061/DISK_YUAN/CS913/mesa/polysomnography/stage_30_minus/mesa-sleep-0006.txt', header=None)

print(len(df_stage))
print(type(df_stage))

1200
<class 'pandas.core.frame.DataFrame'>


In [46]:
np_stage = np.loadtxt('/run/media/u2212061/DISK_YUAN/CS913/mesa/polysomnography/stage_30_minus/mesa-sleep-0006.txt')

print(len(np_stage))
print(type(np_stage))

1200
<class 'numpy.ndarray'>


In [16]:
print(torch.__version__)

2.2.1+cu121


In [159]:
def check_empty_or_na_entries(input_dir):
    for csv_file in tqdm(os.listdir(input_dir), desc='Checking for empty entries'):

        # Read the CSV file into a DataFrame
        df = pd.read_csv(os.path.join(input_dir, csv_file), sep='\t')
        
        # Check for empty entries (NaNs)
        empty_entries = df.isna().sum().sum()
        
        # Check for rows with all NA entries
        rows_all_na = df.isna().all(axis=1).sum()
        
        # Check for columns with all NA entries
        cols_all_na = df.isna().all(axis=0).sum()
        
        if empty_entries != 0 or rows_all_na != 0 or cols_all_na != 0:
            print(f'File: {csv_file}:')
            print(f'Total empty entries: {empty_entries}')
            print(f'Rows with all NA entries: {rows_all_na}')
            print(f'Columns with all NA entries: {cols_all_na}')

100%|██████████| 2055/2055 [14:14<00:00,  2.40it/s]


In [ ]:
# # For mac
# input_dir = '/Users/zhaojingyuan/Desktop/CS913/mesa/polysomnography/edfs'
# output_dir = "/Users/zhaojingyuan/Desktop/CS913/mesa/polysomnography/edfs_ppg_txt"

# convert_edf_to_txt(input_dir, output_dir)